In [4]:
import pandas as pd
import numpy as np
import pyarrow 

In [5]:
import seaborn as sns

### 1. Extracción de datos
Leer el archivo reservas_hoteles.parquet para obtener la información relevante de hoteles de la competencia.

In [50]:
df_eraw = pd.read_parquet('../data/reservas_hoteles.parquet')
df_eraw

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [70]:
df = df_eraw.copy()

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


Pasamos a formato fecha todas las columnas de fechas que están en otros formatos.

In [53]:
def tran_data(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col])
    return dataframe

In [54]:
col_fechas = ["fecha_reserva", "inicio_estancia", "final_estancia"]
df = tran_data(col_fechas, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

### Duplicados

Borramos todos las filas que están duplicadas y son exactamente iguales.

In [55]:
df.duplicated().sum()

np.int64(98)

In [56]:

df.drop_duplicates(inplace=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [57]:
df["id_cliente"].duplicated().sum()

np.int64(153)

### Nulos

Vamos a estudiar cómo son los nulos para ver cómo trabajamos con ellos.

In [63]:
print(df["ciudad"].isnull().sum())  # Contamos los nulos.
print(df["ciudad"].unique())  # Muestra los valores únicos

0
['' 'Madrid']


In [64]:
print(df["ciudad"].isnull().sum())  # Ahora debe mostrar la cantidad correcta de nulos
print(df["ciudad"].unique())  # Ver los valores únicos

0
['' 'Madrid']


In [ ]:
df.replace({"ciudad": ""}, np.nan, inplace=True)
df.fillna({"ciudad": "Madrid"}, inplace=True)

In [76]:
df.sample(20)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
14065,fb612e16-dfd3-4386-9dff-78052e5a0725,a29cc1fa-c6fe-4861-a7e4-95ae1867671d,Isidoro,Clemente,isidoro.clemente@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid
4292,1aa3b259-76d3-4fa4-8953-be75fe181818,aa844588-f2e8-4a8e-b3ea-c1dcda150a39,Sol,Gonzalez,sol.gonzalez@example.com,True,,2025-03-01,2025-03-02,131,NaN,,NaN,Madrid
10645,a67ea469-4b8c-4118-b9ef-5b611f4dc7fb,2107e03b-7aa9-417b-9204-62bffdae5acc,Jaime,León,jaime.león@example.com,False,2025-02-12,2025-03-01,2025-03-02,36,247.05,Hotel Maravilla Real,4.0,Madrid
10312,9181e6b5-e998-418b-81ea-c59f6f09ddbe,cbf4df30-7109-4a51-8c58-5c8022d3823f,Juan Pablo,Balaguer,juan pablo.balaguer@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid
11970,78f48a5f-35e2-4b1f-b64a-750ff9b4571b,a889810d-7331-406d-b51a-07c2ac8da39e,Ceferino,Valls,ceferino.valls@example.com,False,2025-02-06,2025-03-01,2025-03-02,13,245.61,Gran Hotel Madrid,5.0,Madrid
8229,8a853210-8fd5-4a89-bb7d-b635222071ff,9084e099-396d-4c11-8677-49d91183cf03,Nydia,Baños,nydia.baños@example.com,False,2025-02-06,2025-03-01,2025-03-02,14,414.19,Hotel Brisas del Mar,4.0,Madrid
13875,31f72a56-f94b-419b-b06b-4badacd038fe,7297e104-d35c-4a75-8da5-e5b6b06ce78e,Ana,Agustí,ana.agustí@example.com,True,,2025-03-01,2025-03-02,128,NaN,,NaN,Madrid
6369,dbbac288-6b1d-4871-ae10-bc24b7a85396,04ed21c8-6074-4924-9f25-e6df8e776693,Eladio,Castañeda,eladio.castañeda@example.com,False,2025-02-11,2025-03-01,2025-03-02,47,93.03,Hotel Jardines del Rey,5.0,Madrid
3638,13998637-0c16-45d1-a238-f45ee9d2266a,dc97808c-7de4-4377-a49f-6aa31e24f15a,Clotilde,Pujol,clotilde.pujol@example.com,False,2025-02-11,2025-03-01,2025-03-02,28,114.70,Hotel Las Estrellas,2.0,Madrid
12053,c72d8f33-8305-4bbd-bb52-df83f8fd4160,d69f33d7-f9c4-4b11-9226-755bde951266,Marcia,Tudela,marcia.tudela@example.com,True,,2025-03-01,2025-03-02,131,NaN,,NaN,Madrid


In [80]:
df["precio_noche"].describe().T

count    9874.000000
mean      275.117720
std       129.405972
min        50.010000
25%       163.120000
50%       273.315000
75%       387.830000
max       499.820000
Name: precio_noche, dtype: float64

1.	La media (275.12) y la mediana (273.32) son muy cercanas, lo cuál quiere decir que la distribución no está muy sesagada.

2.	La desviación estándar (129.41) es alta, por lo que hay bastante dispersión en los precios.

3.	Los cuartiles muestran una distribución bastante amplia:

	•	25% de los hoteles tienen precios ≤163.12

	•	50% de los hoteles tienen precios ≤273.32
	
	•	75% de los hoteles tienen precios ≤387.83